# Staggered Mesh Method for PBC Exact Exchange.
This is a test file to demonstrate how to use PBC Exact exchange both for hybrid DFT, with diamond and H2 as test materials

## 1. Import packages and build cells

In [13]:
import numpy as np
import pyscf.pbc.scf as pbcscf
import pyscf.pbc.df as df
import khf_stagger
from pyscf.pbc import gto as pbcgto

from pyscf.pbc import dft as pbcdft
from pyscf.pbc.dft import numint as pbcnumint
from pyscf import dft
from pyscf.dft import numint

'''
Hydrogen dimer
'''
def build_h2_cell(nks = (1,1,1),kecut=100,vac_dim=6.0,wrap_around=True):
    cell = pbcgto.Cell()
    cell.unit = 'Bohr'
    cell.atom='''
        H 0.00 0.00 0.00
        H 0.00 0.00 1.80
        '''
    cell.a = np.eye(3)*vac_dim

    cell.verbose = 7
    cell.spin = 0
    cell.charge = 0

    
    
    cell.basis = 'gth-szv'
    cell.pseudo = 'gth-pbe'
    
    cell.ke_cutoff = kecut
    cell.max_memory = 1000
    cell.precision = 1e-8

    cell.build()
    kpts = cell.make_kpts(nks, wrap_around=wrap_around)    
    return cell, kpts


'''
Diamond 
'''
def build_diamond_cell(nks = (1,1,1),kecut=100,wrap_around=True,with_gamma_point=True):
    cell = pbcgto.Cell()
    cell.unit = 'Bohr'
    cell.atom='''
         C 0.0 0.0 0.0
         C 1.68516327271508 1.68516327271508 1.68516327271508
        '''
    cell.a = '''
         0.0 3.370326545430162 3.370326545430162
         3.370326545430162 0.0 3.370326545430162
         3.370326545430162 3.370326545430162 0.0
        '''
    cell.verbose = 7
    cell.spin = 0
    cell.charge = 0
    cell.basis = {'C':'gth-szv'}
    cell.precision = 1e-8
    cell.pseudo = 'gth-pbe'
    cell.ke_cutoff = kecut
    cell.max_memory = 1000

    cell.build()
    cell.omega = 0
    kpts = cell.make_kpts(nks, wrap_around=wrap_around,with_gamma_point=with_gamma_point)    
    return cell, kpts



## 2a. Run HF

In [ ]:
# HF calcuation to base Non-SCF and Split-SCF staggered mesh calculations on.
nks = [2, 2, 2]
cell, kpts = build_h2_cell(nks=nks,kecut=100,vac_dim=6.0,wrap_around=True)

kmf = pbcscf.KRHF(cell, kpts, exxdiv='ewald')
kmf.with_df = df.GDF(cell, kpts).build()
ehf = kmf.kernel()

## 2b. Run Staggered Mesh for HF

In [ ]:
# Staggered 

'''
KHF Stagger, Non-SCF version
Compute densities at shifted mesh non-SCF using F_unshifted. Additional cost
is ~ 1 extra K-build.
'''
kmf_stagger = khf_stagger.KHF_stagger(kmf,"non-scf")
kmf_stagger.kernel()
etot = kmf_stagger.e_tot
ek_stagger = kmf_stagger.ek

print('Non-SCF Stagger')
print('Total energy: ', etot)
print('Exchange energy: ', ek_stagger)

## 3a. Run PBE0

In [ ]:
# Setup
nks = [2, 2, 2]
cell, kpts= build_diamond_cell(nks=nks,kecut=56,with_gamma_point=False)
cell.dimension = 3
cell.build()
Nk = np.prod(nks)

# DFT kernel
dft.numint.NumInt.libxc = dft.xcfun
xc = "PBE0"
krks = pbcdft.KRKS(cell, kpts)
krks.xc = xc
krks.exxdiv = "ewald"
krks.with_df = df.GDF(cell, kpts).build()
edft = krks.kernel()

## 3b. Run Staggered Mesh for EXX term of PBE0

In [ ]:
krks_stagger = khf_stagger.KHF_stagger(krks, "non-scf")
krks_stagger.kernel()
etot = krks_stagger.e_tot
ek_stagger = krks_stagger.ek

print("Non-SCF Stagger with DFT")
print("Total energy: ", etot)
print("Exchange energy: ", ek_stagger)